# Семинар 13 - Ранжирование на деревянных моделях

In [ ]:
import math
from typing import List

import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler

from utils import dcg, ndcg

seed = 42
np.random.seed(seed)

# ListNet


Вспомним реализацию ListNet из прошлого семинара. Перенесем саму модель и инициализации ее весов.

In [ ]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features: int, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )

    def forward(self, input_1: torch.Tensor) -> torch.Tensor:
        logits = self.model(input_1)

        return logits
    
    
def init_weights(module):
    if isinstance(module, torch.nn.Embedding):
        module.weight.data.normal_(mean=0.0, std=1.0)
        if module.padding_idx is not None:
            module.weight.data[module.padding_idx].zero_()
    elif isinstance(module, torch.nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
        
        
def create_model(listnet_num_input_features: int, listnet_hidden_dim: int) -> torch.nn.Module:
    torch.manual_seed(0)
    net = ListNet(listnet_num_input_features, listnet_hidden_dim)
    init_weights(net)

    return net

## Подготовка данных

Обчение модели рандирования проведем на уменьшенной версии набора данных Microsoft Learning to Rank. Этот набор данных является уменьшенной версией набора данных msrank.

Набор обучающих данных содержит 10000 объектов. Каждый объект описывается 138 колонками. Первый столбец содержит значение метки, второй — идентификатор группы объекта (GroupId). Все остальные столбцы содержат характеристики объектов.

Валидационный набор данных содержит 10000 объектов. Структура идентична обучающему набору данных.

Данные загрузим из ```catboost```.

In [ ]:
def get_data() -> List[np.ndarray]:
    train_df, test_df = msrank_10k()

    X_train = train_df.drop([0, 1], axis=1).values
    y_train = train_df[0].values
    query_ids_train = train_df[1].values.astype(int)

    X_test = test_df.drop([0, 1], axis=1).values
    y_test = test_df[0].values
    query_ids_test = test_df[1].values.astype(int)

    return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

In [ ]:
X_train, y_train, query_ids_train, X_test, y_test, query_ids_test = get_data()

In [ ]:
X_train.shape, query_ids_train.shape

In [ ]:
np.unique(query_ids_train)

In [ ]:
X_train.mean(), X_train.std()

Проведем подготовку данных для обучения. Видно, что среднее и дисперсия в данных не идеальна. Исправим это для более устойчивого обучения модели. 

Затем подготовим данные для примения в обучении модели на ```torch```.

In [ ]:
def scale_features_in_query_groups(inp_feat_array: np.ndarray, inp_query_ids: np.ndarray) -> np.ndarray:
    # your code here
    # scale each data by query
    for id in np.unique(inp_query_ids):
        pass

    return inp_feat_array


def prepare_data() -> List[np.ndarray]:
    X_train, y_train, query_ids_train, X_test, y_test, query_ids_test = get_data()
    # your code here: 
    # 1. scale train and test data 
    # 2. convert data to torch
    X_train = None
    ys_train = None

    X_test = None
    ys_test = None
    
    return X_train, ys_train, query_ids_train, X_test, ys_test, query_ids_test

In [ ]:
X_train, ys_train, query_ids_train, X_test, ys_test, query_ids_test = prepare_data()

## Подготовка этапов обучения

In [ ]:
def ndcg_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, ndcg_top_k: int) -> float:
    try:
        return ndcg(ys_true, ys_pred, gain_scheme='exp2', top_k=ndcg_top_k)
    except ZeroDivisionError:
        return float(0)

In [ ]:
def calc_loss(batch_ys: torch.FloatTensor, batch_pred: torch.FloatTensor) -> torch.FloatTensor:
    P_y_i = torch.softmax(batch_ys, dim=0)
    P_z_i = torch.softmax(batch_pred, dim=0)

    return -torch.sum(P_y_i * torch.log(P_z_i))

In [ ]:
n_epochs: int = 5
listnet_hidden_dim: int = 30
lr: float = 0.001
ndcg_top_k: int = 10
num_input_features = X_train.shape[1]


model = create_model(num_input_features, listnet_hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
def _train_one_epoch(model, optimizer, X_train, ys_train, query_ids_train) -> None:
    # your code here
    pass

In [ ]:
_train_one_epoch(model, optimizer, X_train, ys_train, query_ids_train)

In [ ]:
def _eval_test_set(model, X_test, ys_test, query_ids_test) -> float:
    # your code here
    ndcgs = []

    return np.mean(ndcgs)

In [ ]:
_eval_test_set(model, X_test, ys_test, query_ids_test)

In [ ]:
def fit(n_epochs, model, optimizer, X_train, ys_train, query_ids_train, X_test, ys_test, query_ids_test) -> List[float]:
    val_ndcg = []
    
    for epoch in tqdm(range(n_epochs)):
        _train_one_epoch(model, optimizer, X_train, ys_train, query_ids_train)
        val_metric = _eval_test_set(model, X_test, ys_test, query_ids_test)

        val_ndcg.append(val_metric)

    return val_ndcg

In [ ]:
n_epochs: int = 100
listnet_hidden_dim: int = 10
lr: float = 0.001
ndcg_top_k: int = 10
num_input_features = X_train.shape[1]


model = create_model(num_input_features, listnet_hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


val_ndcg = fit(n_epochs, model, optimizer, X_train, ys_train, query_ids_train, X_test, ys_test, query_ids_test)

In [ ]:
plt.plot(val_ndcg)
plt.xlabel('Epochs')
plt.ylabel('Val/NDCG')
plt.grid()
plt.show()

# LambdaRank

$$\lambda = \left(0.5 * (1 - S_{ij}) - \frac {1} {1 + e^{s_i - s_j}}\right) |\Delta nDCG|$$

$$\Delta nDCG = \frac {1} {IdealDCG} (2^i - 2^j) \left(\frac {1} {log_2(1+i)} - \frac {1} {log_2(1+j)}\right)$$

In [ ]:
# в y_true лежат оценки релевантности
y_true = torch.LongTensor([[5, 3, 2, 5, 1, 1]]).reshape(-1, 1)
y_pred = torch.FloatTensor([3.2, 0.4, -0.1, -2.1, 0.5, 0.01]).reshape(-1, 1)

In [ ]:
y_pred

In [ ]:
def compute_lambdas(y_true, y_pred, ndcg_scheme='exp2'):
    # рассчитаем нормировку, IdealDCG
    ideal_dcg = dcg(y_true, y_true, ndcg_scheme)
    N = 1 / ideal_dcg
    
    # рассчитаем порядок документов согласно оценкам релевантности
    _, rank_order = torch.sort(y_true, descending=True, axis=0)
    rank_order += 1
    
    with torch.no_grad():
        # получаем все попарные разницы скоров в батче
        pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t()))
        
        # поставим разметку для пар, 1 если первый документ релевантнее
        # -1 если второй документ релевантнее
        Sij = compute_labels_in_batch(y_true)
        # посчитаем изменение gain из-за перестановок
        gain_diff = compute_gain_diff(y_true, ndcg_scheme)
        
        # посчитаем изменение знаменателей-дискаунтеров
        decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.t() + 1.0))
        # посчитаем непосредственное изменение nDCG
        delta_ndcg = torch.abs(N * gain_diff * decay_diff)
        # посчитаем лямбды
        lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
        lambda_update = torch.sum(lambda_update, dim=1, keepdim=True)
        
        return Sij, gain_diff, decay_diff, delta_ndcg, lambda_update
    
    
def compute_labels_in_batch(y_true):
    
    # разница релевантностей каждого с каждым объектом
    rel_diff = y_true - y_true.t()
    
    # 1 в этой матрице - объект более релевантен
    pos_pairs = (rel_diff > 0).type(torch.float32)
    
    # 1 тут - объект менее релевантен
    neg_pairs = (rel_diff < 0).type(torch.float32)
    Sij = pos_pairs - neg_pairs
    return Sij


def compute_gain_diff(y_true, gain_scheme):
    if gain_scheme == "exp2":
        gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
    elif gain_scheme == "diff":
        gain_diff = y_true - y_true.t()
    else:
        raise ValueError(f"{gain_scheme} method not supported")
    return gain_diff

In [ ]:
y_pred - y_pred.t()

In [ ]:
y_true - y_true.t()

In [ ]:
Sij, gain_diff, decay_diff, delta_ndcg, lambda_update = compute_lambdas(y_true, y_pred)

In [ ]:
Sij

In [ ]:
gain_diff

In [ ]:
# пример вычисления элемента gain diff для первого (релевантность 5) и последнего документа (1); 
# для первого (5) и второго (3) документа
(2**5 - 1) - (2**1 -1), (2**5 - 1) - (2**3-1)

In [ ]:
decay_diff

In [ ]:
# посчитаем изменение знаменателей-дискаунтеров для первого и последнего документа
(1 / np.log2(1+1)) - (1 / np.log2(1+6))

In [ ]:
delta_ndcg

In [ ]:
lambda_update

In [ ]:
for _ in range(100):
    _, _, _, _, lambda_update = compute_lambdas(y_true, y_pred)
    y_pred -= lambda_update

In [ ]:
rank_indexes = torch.argsort(y_pred, dim=0, descending=True)

In [ ]:
y_pred

In [ ]:
y_true[rank_indexes]

In [ ]:
# полностью правильное ранжирование
torch.sort(y_true, dim=0, descending=True)[0]

In [ ]:
y_true = torch.LongTensor([[5,3,2,5,1,1]]).reshape(-1,1)

# совсем плохие предсказанные скоры в начале
y_pred = torch.FloatTensor([-3.0, 2.0, 3.0, -4.0, 6.0, 8.5]).reshape(-1,1)

In [ ]:
ndcg(y_true, y_pred)

In [ ]:
for _ in range(100):
    _, _, _, _, lambda_update = compute_lambdas(y_true, y_pred)
    y_pred -= lambda_update

In [ ]:
ndcg(y_true, y_pred)

In [ ]:
# полностью правильное ранжирование при увеличении количества итераций
y_pred